In [1]:
import numpy as np

In [5]:
z = np.random.randint(0, 10, (1, 2))
z

array([[1, 9]])

In [ ]:

for move in ['down', 'left', 'left', 'left', 'left', 'left', 'up', 
'up', 'up', 'right', 'right', 'right', 'right', 'right', 'down', 'down', 
'left', 'left', 'left', 'left', 'up', 'up', 'up', 'right', 'right', 'right', 
'right', 'right', 'down', 'down', 'down', 'down', 'down', 'left', 'left', 'left',
'up', 'up', 'up', 'up', 'up', 'left', 'left', 'left', 'left', 'down', 'down', 'down', 
'down', 'down', 'down', 'right', 'right', 'right', 'right', 'down', 'left', 'left', 'left', 
'left', 'left', 'up', 'up', 'up', 'up', 'up', 'right', 'right', 'right', 'right', 'right', 'right', 
'right', 'right', 'right', 'down', 'down', 'down', 'down', 'down', 'left', 'left', 'left', 'left']:
    game.move(move)